In [1]:
! pip install gym --quiet
! pip install highway-env --quiet
! pip install neat-python --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 4.0 MB/s eta 0:00:00


In [2]:
import gymnasium as gym
import highway_env
import matplotlib.pyplot as plt
import time
import torch
import numpy as np
import os
import neat
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [3]:
class DeepQNetwork(nn.Module):
    def __init__(self,lr,input_dims,fc1_dims,fc2_dims,n_actions):
        super(DeepQNetwork,self).__init__()
        self.input_dims=input_dims
        self.fc1_dims=fc1_dims
        self.fc2_dims=fc2_dims
        self.n_actions=n_actions

        self.fc1=nn.Linear(self.input_dims,self.fc1_dims)
        self.fc2=nn.Linear(self.fc1_dims,self.fc2_dims)
        self.fc3=nn.Linear(self.fc2_dims,self.n_actions)

        self.optimizer=optim.Adam(self.parameters(),lr=lr)
        self.loss=nn.MSELoss()
        self.device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self,state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions=self.fc3(x) 
        return actions

class Agent():
    def __init__(self,gamma,epsilon,lr,input_dims,batch_size,n_actions,max_mem_size=100000,eps_end=0.01,eps_dec=5e-4):
        """

        :param gamma: weight of future rewards
        :param epsilon: explore/exploit dilemma ratio
        :param lr: learning rate
        :param batch_size: batch size of memory
        :param n_actions: number of actions
        :param max_mem_size: memory of past episodes
        :param eps_end: once epsilon reaches this, training end
        :param eps_dec: what to decrement epsilon at each timestamp
        """

        self.gamma=gamma
        self.epsilon=epsilon
        self.eps_min=eps_end
        self.eps_dec=eps_dec
        self.lr=lr
        self.action_space=[i for i in range(n_actions)]#List of available actions
        self.mem_size=max_mem_size
        self.batch_size=batch_size
        self.mem_ctr=0 #Counter for array of memory

        self.Q_eval=DeepQNetwork(self.lr,n_actions=n_actions,input_dims=input_dims,fc1_dims=256,fc2_dims=256)

        self.state_memory=np.zeros((self.mem_size,input_dims),dtype=np.float32)
        self.new_state_memory=np.zeros((self.mem_size,input_dims),dtype=np.float32) #Memory for next state

        self.action_memory = np.zeros(self.mem_size,dtype=np.int32) #Memory for actions, stores int for index of action taken
        self.reward_memory = np.zeros(self.mem_size,dtype=np.float32) #Memory for rewards
        self.terminal_memory = np.zeros(self.mem_size,dtype=bool) #Memory for terminal states

    def store_transition(self,state,action,reward,state_,done):
        """
        Function for storing in memory each info about a state-action
        :param state:
        :param action:
        :param reward:
        :param state_:
        :param done:
        :return:
        """
        index=self.mem_ctr % self.mem_size
        self.state_memory[index] = state

        self.new_state_memory[index] = state_
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = done

        self.mem_ctr+=1

    def choose_action(self,observation):
        if np.random.random()>self.epsilon:
            state= torch.tensor(observation).to(self.Q_eval.device)
            actions=self.Q_eval.forward(state)
            action=torch.argmax(actions).item() #Take action with highest activation output
        else:
            action=np.random.choice(self.action_space)
        return action

    def learn(self):
        """
        Once batch size is filled with episodes, learn from the episodes you have in batch
        :return:
        """
        if self.mem_ctr < self.batch_size: #Skip computation until batchsize is full
            return
        self.Q_eval.optimizer.zero_grad()
        max_mem=min(self.mem_ctr,self.mem_size)
        batch = np.random.choice(max_mem,self.batch_size,replace=False)

        batch_index=np.arange(self.batch_size,dtype=np.int32)

        state_batch = torch.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = torch.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
        reward_batch = torch.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch=torch.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)

        action_batch=self.action_memory[batch]

        q_eval = self.Q_eval.forward(state_batch)[batch_index,action_batch]
        q_next = self.Q_eval.forward(new_state_batch)
        q_next[terminal_batch]=0.0

        q_target=reward_batch+self.gamma*torch.max(q_next,dim=1)[0]

        loss=self.Q_eval.loss(q_target,q_eval).to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()

        #Decrease epsilon
        self.epsilon=self.epsilon-self.eps_dec if self.epsilon > self.eps_min else self.eps_min

In [4]:
def renderEnvironmentExample(env):
    obs, info = env.reset()
    env.render()
    env.close()


def list_envs():
    all_envs = gym.envs.registry
    print(sorted(all_envs))

def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        observation = env.reset(seed=10)[0].flatten()
        done = False
        fitness = 0.0
        while not done:
            action = net.activate(observation)
            #argmax the function
            action = np.argmax(action)
            observation_, reward, done, _, info = env.step(action)
            observation_ = observation.flatten()
            fitness += reward
            observation = observation_
        genome.fitness = fitness

def learnDQNetwork(env, config_path):
    agent = Agent(gamma=0.99, epsilon=1.0, batch_size=128, n_actions=5, eps_end=0.01, input_dims=25, lr=5e-4)
    scores, epshistory = [], []

    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                                neat.DefaultSpeciesSet, neat.DefaultStagnation,
                                config_path)
    
    p = neat.Population(config)
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    winner = p.run(eval_genomes, 50)
    print('\nBest genome:\n{!s}'.format(winner))

In [ ]:
print(torch.cuda.is_available())
#list_envs()
env=gym.make("highway-fast-v0", render_mode='rgb_array')
config_path = 'config-ff.txt'

learnDQNetwork(env, config_path)


True

 ****** Running generation 0 ****** 



/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	